# CS 5588 — GenAI Product Starter Notebook (Revised)

This notebook generates a **product-oriented GenAI app skeleton**:
- **Streamlit UI** for a user-facing GenAI product
- **Optional FastAPI** backend (extension)
- **Automatic interaction logging** to `logs/product_metrics.csv`

## What was improved in this revised version
- Adds a **"generated files" verification cell** so students can confirm outputs.
- Adds **CSV-safe logging** (prevents spreadsheet formula injection for common prefixes like `=`).
- Expands the log schema slightly (adds `interaction_id` and `scenario_id`).

## Recommended workflow (CS 5588)
1. Prototype product logic here (prompting/RAG/tool use)
2. Move stable logic into `product_core.py`
3. Build UI in `app.py`
4. (Optional) Serve via `api.py`
5. Deploy and use logs to iterate your product


## 0) Environment setup (recommended)

**Terminal:**
```bash
python -m venv genai_env
source genai_env/bin/activate   # Mac/Linux
genai_env\Scripts\activate      # Windows
pip install -U pip
pip install streamlit fastapi uvicorn requests pandas python-dotenv
```


In [25]:
import os, sys, platform
print('Python:', sys.version)
print('Platform:', platform.platform())
print('Working directory:', os.getcwd())

Python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Platform: Linux-6.6.105+-x86_64-with-glibc2.35
Working directory: /content


## 1) Install packages (skip if installed via terminal)

In managed notebook environments, installs may not persist.


In [26]:
!pip -q install streamlit fastapi uvicorn requests pandas python-dotenv

In [31]:
!pip install streamlit pyngrok

In [32]:
!ngrok config add-authtoken 39VrbVGFFud7BOwY1k4PTa7J0EA_e5YBTjU3N7w9mcNK9PGG

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [33]:
!streamlit run app.py &>/content/logs.txt &


## 2) Generate CS 5588 product skeleton files

This writes:
- `product_core.py` (core product logic)
- `app.py` (Streamlit product UI)
- `api.py` (FastAPI backend; optional extension)
- `requirements.txt`
- `README.md`

### Product logging
Every interaction appends one row to:
`logs/product_metrics.csv`


In [30]:
# product_core = '''
"""CS 5588 Product Core

Replace `generate_response` with your project logic:
- Prompting (system + user)
- RAG retrieval + grounded generation
- Tool use / function calling
- Multimodal processing (if applicable)
"""

from typing import Dict, Any, List
import time

def generate_response(user_input: str, mode: str = "assistant") -> Dict[str, Any]:
    """Return a product-style response payload."""
    t0 = time.time()

    # TODO: Replace with your actual pipeline.
    output_text = f"[Demo output | mode={mode}] You said: {user_input}"
    sources: List[str] = []  # populate with evidence IDs when you add retrieval
    failure_flag = False

    runtime_ms = int((time.time() - t0) * 1000)
    return {
        "output_text": output_text,
        "sources": sources,
        "runtime_ms": runtime_ms,
        "failure_flag": failure_flag,
        "safety_or_policy_note": "",
    }
# '''

# app_py = '''
import os
import uuid
import pandas as pd
import streamlit as st
from product_core import generate_response

st.set_page_config(page_title="CS 5588 GenAI Product", layout="centered")

def csv_safe_text(s: str) -> str:
    """Basic CSV/Spreadsheet safety: prevent formula injection when opened in Excel/Sheets."""
    if not isinstance(s, str):
        s = str(s)
    s = s.strip()
    if s[:1] in ("=", "+", "-", "@"):
        return "'" + s
    return s

st.title("CS 5588 — GenAI Product Prototype")
st.caption("A product-style GenAI app skeleton with logging for iteration.")

# --- Product controls ---
mode = st.selectbox("Product Mode", ["assistant", "summarize", "recommend"], index=0)
scenario_id = st.selectbox(
    "Scenario ID (for evaluation)",
    ["S1", "S2", "S3", "S4", "S5"],
    index=0,
)
user_input = st.text_area("User Input", height=120, placeholder="Describe what you want the product to do...")

# Simple user rating signal for product iteration
rating = st.slider("User rating (for iteration)", min_value=1, max_value=5, value=4)

if st.button("Run"):
    if not user_input.strip():
        st.warning("Please enter a user input.")
    else:
        interaction_id = str(uuid.uuid4())
        payload = generate_response(user_input=user_input.strip(), mode=mode)

        st.subheader("Output")
        st.write(payload.get("output_text", ""))

        st.subheader("Evidence / Sources (if applicable)")
        sources = payload.get("sources", [])
        if sources:
            st.write(sources)
        else:
            st.info("No sources returned (add retrieval to populate this).")

        st.subheader("Product Metrics")
        runtime_ms = payload.get("runtime_ms", None)
        failure_flag = payload.get("failure_flag", False)
        st.write({
            "interaction_id": interaction_id,
            "scenario_id": scenario_id,
            "runtime_ms": runtime_ms,
            "failure_flag": failure_flag,
            "user_rating": rating,
            "mode": mode,
        })

        # --- Logging ---
        os.makedirs("logs", exist_ok=True)
        log_path = os.path.join("logs", "product_metrics.csv")

        safe_input = csv_safe_text(
            user_input.strip().replace("\n", " ")
        )[:500]

        row = {
            "timestamp": pd.Timestamp.utcnow().isoformat(),
            "interaction_id": interaction_id,
            "scenario_id": scenario_id,
            "mode": mode,
            # "user_input": csv_safe_text(user_input.strip().replace("\n", " "))[:500],
            "user_input": safe_input,
            "runtime_ms": runtime_ms,
            "failure_flag": bool(failure_flag),
            "user_rating": int(rating),
            "sources": csv_safe_text(str(sources))[:500],
        }

        df = pd.DataFrame([row])
        if os.path.exists(log_path):
            df.to_csv(log_path, mode="a", header=False, index=False)
        else:
            df.to_csv(log_path, index=False)

        st.success(f"Logged interaction to {log_path}")

st.divider()
st.caption("Deployment tip: Streamlit Cloud expects an entrypoint like app.py or app/main.py.")


# api_py = '''
from fastapi import FastAPI
from pydantic import BaseModel
from product_core import generate_response

app = FastAPI(title="CS 5588 Product API")

class ProductRequest(BaseModel):
    user_input: str
    mode: str = "assistant"

@app.get("/")
def health():
    return {"status": "ok", "message": "CS 5588 Product API running"}

@app.post("/run")
def run(req: ProductRequest):
    return generate_response(user_input=req.user_input, mode=req.mode)
# '''

requirements = '''
streamlit
fastapi
uvicorn
pydantic
requests
pandas
python-dotenv
'''.lstrip()

readme = '''
# CS 5588 — GenAI Product Prototype

This repo contains a CS 5588 product-style GenAI app skeleton:
- Streamlit UI (`app.py`)
- Optional FastAPI backend (`api.py`)
- Product logging to `logs/product_metrics.csv`

## Setup
```bash
python -m venv genai_env
source genai_env/bin/activate   # Mac/Linux
genai_env\\Scripts\\activate      # Windows
pip install -U pip
pip install -r requirements.txt
```

## Run Streamlit (required)
```bash
streamlit run app.py
```
Open: http://localhost:8501

## Run FastAPI (optional extension)
```bash
uvicorn api:app --reload
```
Docs: http://127.0.0.1:8000/docs

## Logs
Each interaction appends to:
- `logs/product_metrics.csv`

## What to customize
Edit `product_core.generate_response()` to implement your product logic (prompting/RAG/tools).
'''.lstrip()

for name, content in [
    ("product_core.py", product_core),
    ("app.py", app_py),
    ("api.py", api_py),
    ("requirements.txt", requirements),
    ("README.md", readme),
]:
    with open(name, 'w', encoding='utf-8') as f:
        f.write(content)

print('Wrote: product_core.py, app.py, api.py, requirements.txt, README.md')

2026-02-12 22:39:49.635 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-12 22:39:49.642 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-12 22:39:49.654 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-12 22:39:49.658 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-12 22:39:49.668 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-12 22:39:49.673 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-12 22:39:49.682 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-12 22:39:49.687 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

Wrote: product_core.py, app.py, api.py, requirements.txt, README.md


## 2.1) Verify generated files (recommended)

Run this after file generation to confirm everything was created in the correct folder.


In [28]:
import os
expected = ['product_core.py', 'app.py', 'api.py', 'requirements.txt', 'README.md']
missing = [f for f in expected if not os.path.exists(f)]
print('Expected files:', expected)
print('Missing files:', missing)
print('Current folder files (top 30):', sorted(os.listdir('.'))[:30])
assert not missing, f"Missing files: {missing}"
print('✅ All expected files are present')

Expected files: ['product_core.py', 'app.py', 'api.py', 'requirements.txt', 'README.md']
Missing files: []
Current folder files (top 30): ['.config', 'README.md', 'api.py', 'app.py', 'logs.txt', 'product_core.py', 'requirements.txt', 'sample_data']
✅ All expected files are present


## 3) Quick correctness test (no server needed)

This checks that `generate_response()` returns the expected payload structure.


In [29]:
from product_core import generate_response

payload = generate_response('Test input', mode='assistant')
print(payload)

assert 'output_text' in payload
assert 'runtime_ms' in payload
assert 'failure_flag' in payload
assert isinstance(payload.get('sources', []), list)
print('✅ product_core.generate_response() looks correct')

{'output_text': '[Demo output | mode=assistant] You said: Test input', 'sources': [], 'runtime_ms': 0, 'failure_flag': False, 'safety_or_policy_note': ''}
✅ product_core.generate_response() looks correct


## 4) Run instructions (terminal recommended)

### Run Streamlit (required)
```bash
streamlit run app.py
```

### Run FastAPI (optional extension)
```bash
uvicorn api:app --reload
```


In [34]:
from pyngrok import ngrok

public_url = ngrok.connect(8501)
print("Streamlit URL:", public_url)


Streamlit URL: NgrokTunnel: "https://unshrunken-jerilyn-concussant.ngrok-free.dev" -> "http://localhost:8501"


## 5) Suggested submission structure (CS 5588)

```
.
├── CS5588_GenAI_Product_Starter_Revised.ipynb
├── product_core.py
├── app.py
├── api.py                 # optional extension
├── requirements.txt
├── README.md
└── logs/
    └── product_metrics.csv
```
